<a href="https://colab.research.google.com/github/Tunji17/huggingface-tasks/blob/main/SFT_Qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets unsloth transformers torch matplotlib --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.6/191.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import wandb
import torch
import random
from unsloth import FastLanguageModel
from transformers import pipeline
from datasets import load_dataset, DatasetDict

dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train")

random_int = random.randint(0, len(dataset))
random_idx = dataset[random_int: random_int + 1]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

(…)-00000-of-00001-8b6e212f3e1ece96.parquet:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [4]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2nji (_2nji) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [6]:
prompt_format = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Output:
{}"""
EOS_TOKEN = tokenizer.eos_token

In [7]:
train_test_split = dataset.train_test_split(test_size=0.2)
test_val_split = train_test_split["test"].train_test_split(test_size=0.2)
dataset = DatasetDict(
    {
        "train": train_test_split["train"],
        "test": test_val_split["train"],
        "validation": test_val_split["test"],
    }
)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 14889
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 2978
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 745
    })
})


In [8]:
def format_prompt(examples, with_output=True):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    prompts = []
    print(len(instructions), len(inputs), len(outputs))
    if with_output:
        for instruction, input, output in zip(instructions, inputs, outputs):
            prompt = prompt_format.format(instruction, input, output) + EOS_TOKEN
            prompts.append(prompt)
    else:
        for instruction, input in zip(instructions, inputs):
            prompt = prompt_format.format(instruction, input, "") + EOS_TOKEN
            prompts.append(prompt)
    return {"prompts": prompts}

In [9]:
from transformers import TextStreamer
FastLanguageModel.for_inference(model)

inputs = tokenizer(format_prompt(random_idx, False)["prompts"],
                   return_tensors="pt", padding=True, truncation=True).to("cuda")

text_streamer = TextStreamer(tokenizer)
inference = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64, use_cache = True)
outputs = tokenizer.batch_decode(inference)

for item in outputs:
    print("Generated Response: \n", item)


1 1 1
<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Can you develop a Python program to implement a stack data structure?

### Input:


### Output:
<|end_of_text|><|begin_of_text|>://This is a sample output.
<|end_of_text|>
Generated Response: 
 <|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Can you develop a Python program to implement a stack data structure?

### Input:


### Output:
<|end_of_text|><|begin_of_text|>://This is a sample output.
<|end_of_text|>


In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42, # seed
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [11]:
dataset = dataset.map(format_prompt, batched=True)

Map:   0%|          | 0/14889 [00:00<?, ? examples/s]

1000 1000 1000
1000 1000 1000
1000 1000 1000
1000 1000 1000
1000 1000 1000
1000 1000 1000
1000 1000 1000
1000 1000 1000
1000 1000 1000
1000 1000 1000
1000 1000 1000
1000 1000 1000
1000 1000 1000
1000 1000 1000
889 889 889


Map:   0%|          | 0/2978 [00:00<?, ? examples/s]

1000 1000 1000
1000 1000 1000
978 978 978


Map:   0%|          | 0/745 [00:00<?, ? examples/s]

745 745 745


In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    dataset_text_field = "prompts",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        run_name = "sft_qlora_Meta-Llama-3.1-8B",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        hub_model_id = "2nji/sft_qlora_Meta-Llama-3.1-8B",
        hub_strategy="every_save",  # Options: "end", "every_save", "checkpoint"
        push_to_hub=True,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 1000,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = ["wandb"]
    ),
)

Tokenizing to ["prompts"] (num_proc=2):   0%|          | 0/14889 [00:00<?, ? examples/s]

Tokenizing to ["prompts"] (num_proc=2):   0%|          | 0/2978 [00:00<?, ? examples/s]

In [13]:
wandb.init(project="huggingface", name="sft_qlora_Meta-Llama-3.1-8B")

wandb.watch(model, log="all")

In [14]:
# @title Show memory stats before training run
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.887 GB of memory reserved.


In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 14,889 | Num Epochs = 1 | Total steps = 1,861
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/4,670,623,744 (0.90% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.354900
2,1.584100
3,1.382500
4,1.449600
5,1.242500
6,1.032900
7,0.951800
8,0.917400
9,0.501000
10,0.892100


In [16]:
# @title Show memory and time stats after training run
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

15886.1758 seconds used for training.
264.77 minutes used for training.
Peak reserved memory = 8.586 GB.
Peak reserved memory for training = 2.699 GB.
Peak reserved memory % of max memory = 58.246 %.
Peak reserved memory for training % of max memory = 18.309 %.


In [18]:
trainer.push_to_hub("Saving model and tokenizer")

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/2nji/sft_qlora_Meta-Llama-3.1-8B/commit/cf1c5739ae1a59db46e440ab9f3aaaced0978db9', commit_message='Saving model and tokenizer', commit_description='', oid='cf1c5739ae1a59db46e440ab9f3aaaced0978db9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/2nji/sft_qlora_Meta-Llama-3.1-8B', endpoint='https://huggingface.co', repo_type='model', repo_id='2nji/sft_qlora_Meta-Llama-3.1-8B'), pr_revision=None, pr_num=None)

In [19]:
random_int = random.randint(0, len(dataset["validation"]))
random_idx = dataset["validation"][random_int: random_int + 1]

In [20]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(format_prompt(random_idx, False)["prompts"],
                   return_tensors="pt", padding=True, truncation=True).to("cuda")

text_streamer = TextStreamer(tokenizer)
inference = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64, use_cache = True)
outputs = tokenizer.batch_decode(inference)

for item in outputs:
    print("Generated Response: \n", item)

1 1 1
<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Develop a Python 3 program that takes a list of integers as an argument and returns the mean of the list.

### Input:
[1,2,3,4,5]

### Output:
<|end_of_text|><|begin_of_text|>:// Function to calculate the mean of a list of integers
def mean(list):
    sum = 0
    for x in list:
        sum += x
    return sum/len(list)

# Main
list = [1,2,3,4,5]
print(mean(list))<|end_of_text|>
Generated Response: 
 <|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Develop a Python 3 program that takes a list of integers as an argument and returns the mean of the list.

### Input:
[1,2,3,4,5]

### Output:
<|end_of_text|><|begin_of_text|>:// Function to calculate the mean of a list of integers
def mean(list):
    sum = 0
    for x in list:
        sum += x
    return su